In [2]:
import pandas as pd
import time
import re
import tweepy
import requests
from bs4 import BeautifulSoup
from types import MappingProxyType
import datetime
import math
import random
from urllib.request import urlopen
import numpy as np
# import modules.settings as Settings

# ローマ字⇒カナ変換したい

# 背番号変換したい

In [6]:
def scrapeFbrefStandardStats0():
    # スクレイピング対象のURLを生成する
    url = 'https://fbref.com/en/squads/b8fd03ef/2022-2023/all_comps/Manchester-City-Stats-All-Competitions'
    
    # スクレイピング
    html = urlopen(url).read().decode('utf-8')

    df = pd.read_html(html)[0]
    
    # マルチインデックスを解消する
    df = df.reset_index(drop=True)
    
    print(df.columns)

    # 必要なカラムのみを抽出する
    df = df[[( 'Unnamed: 0_level_0',   'Player'),
             ( 'Unnamed: 2_level_0',      'Pos'),
             ( 'Unnamed: 4_level_0',       'MP'),
             (        'Performance',      'Gls'),
             (        'Performance',      'Ast')
            ]]
    return df

df2 = scrapeFbrefStandardStats0()
df2

MultiIndex([( 'Unnamed: 0_level_0',   'Player'),
            ( 'Unnamed: 1_level_0',   'Nation'),
            ( 'Unnamed: 2_level_0',      'Pos'),
            ( 'Unnamed: 3_level_0',      'Age'),
            ( 'Unnamed: 4_level_0',       'MP'),
            (       'Playing Time',   'Starts'),
            (       'Playing Time',      'Min'),
            (       'Playing Time',      '90s'),
            (        'Performance',      'Gls'),
            (        'Performance',      'Ast'),
            (        'Performance',      'G+A'),
            (        'Performance',     'G-PK'),
            (        'Performance',       'PK'),
            (        'Performance',    'PKatt'),
            (        'Performance',     'CrdY'),
            (        'Performance',     'CrdR'),
            (           'Expected',       'xG'),
            (           'Expected',     'npxG'),
            (           'Expected',      'xAG'),
            (           'Expected', 'npxG+xAG'),
            (       

Unnamed: 0_level_0 Unnamed: 2_level_0 Unnamed: 4_level_0 Performance  \
                 Player                Pos                 MP         Gls   
0                 Rodri              MF,DF                 41         2.0   
1               Ederson                 GK                 36         0.0   
2        Erling Haaland                 FW                 37        42.0   
3       Kevin De Bruyne                 MF                 37         7.0   
4        İlkay Gündoğan                 MF                 38         5.0   
5         Manuel Akanji                 DF                 33         0.0   
6        Bernardo Silva              MF,FW                 40         4.0   
7         Jack Grealish              FW,MF                 37         4.0   
8            Rúben Dias                 DF                 31         1.0   
9            Nathan Aké                 DF                 33         2.0   
10         Riyad Mahrez              FW,MF                 35        12.0   
11           Phil Foden              FW,MF                 36        13.0   
12         João Cancelo                 DF                 26         2.0   
13       Julián Álvarez                 FW                 35        13.0   
14          John Stones              DF,MF                 21         1.0   
15          Kyle Walker                 FW                 23         0.0   
16      Aymeric Laporte                 DF                 16         0.0   
17           Rico Lewis                 DF                 18         1.0   
18        Stefan Ortega                 GK                  9         0.0   
19         Sergio Gómez              DF,MF                 16         0.0   
20          Cole Palmer              MF,FW                 20         1.0   
21      Kalvin Phillips                 MF                 13         0.0   
22  Josh Wilson-Esbrand                 DF                  2         0.0   
23       Máximo Perrone                 MF                  2         0.0   
24         Scott Carson                 GK                  0         NaN   
25  Alexander Robertson                 MF                  0         NaN   
26         Shea Charles                 MF                  0         NaN   
27        Claudio Gomes                 MF                  0         NaN   
28           Ben Knight                 MF                  0         NaN   
29           Liam Delap                 FW                  0         NaN   
30      Luke Mbete-Tatu                 DF                  0         NaN   

          
     Ast  
0    6.0  
1    1.0  
2    5.0  
3   20.0  
4    4.0  
5    1.0  
6    5.0  
7    8.0  
8    0.0  
9    0.0  
10   8.0  
11   7.0  
12   5.0  
13   4.0  
14   2.0  
15   1.0  
16   0.0  
17   0.0  
18   0.0  
19   2.0  
20   0.0  
21   0.0  
22   0.0  
23   0.0  
24   NaN  
25   NaN  
26   NaN  
27   NaN  
28   NaN  
29   NaN  
30   NaN

In [18]:
def scrapeFbrefStandardStats1():
    # スクレイピング対象のURLを生成する
    url = 'https://fbref.com/en/squads/b8fd03ef/2022-2023/all_comps/Manchester-City-Stats-All-Competitions'
    
    # スクレイピング
    html = urlopen(url).read().decode('utf-8')

    df = pd.read_html(html)[0]

    # マルチインデックスを解消する
    df = df.reset_index(drop=True)
    
    # 欠損値を含む行を削除する
    df = df.dropna()
    
    # 欠損データパッチ
    df.loc[df['Unnamed: 0_level_0',   'Player'] == 'Kyle Walker', ('Unnamed: 2_level_0', 'Pos')] = 'DF'

    # ポジションを優先順位に従って変換する
    pos_order = {'FW': 0, 'MF': 1, 'DF': 2, 'GK': 3}
    df['Pos'] = df[('Unnamed: 2_level_0', 'Pos')].str.split(',').str[0].str.strip().map(pos_order)

    # ポジションで並べ替える
    df = df.sort_values(by=[('Pos', ''), ( 'Unnamed: 4_level_0',       'MP')], ascending=[True, False]).reset_index(drop=True)

    # 必要なカラムのみを抽出する
    df = df[[( 'Unnamed: 0_level_0',   'Player'),
             ( 'Unnamed: 2_level_0',      'Pos'),
             ( 'Unnamed: 4_level_0',       'MP'),
             (        'Performance',      'Gls'),
             (        'Performance',      'Ast')
            ]]

    # 列名を変更する
    df.columns = ['Player', 'Pos', 'MP', 'Gls', 'Ast']

    # Gls列とAst列を整数型に変換し、小数点以下を切り捨てる
    # 欠損値と無限大の値を除外して整数に変換する
    df['Gls'] = df['Gls'].replace([np.inf, -np.inf], np.nan).dropna().astype(int)
    df['Ast'] = df['Ast'].replace([np.inf, -np.inf], np.nan).dropna().astype(int)

    print(df[:10])

    return df

df2 = scrapeFbrefStandardStats1()
df2

            Player    Pos  MP  Gls  Ast
0   Erling Haaland     FW  37   42    5
1    Jack Grealish  FW,MF  37    4    8
2       Phil Foden  FW,MF  36   13    7
3     Riyad Mahrez  FW,MF  35   12    8
4   Julián Álvarez     FW  35   13    4
5            Rodri  MF,DF  41    2    6
6   Bernardo Silva  MF,FW  40    4    5
7   İlkay Gündoğan     MF  38    5    4
8  Kevin De Bruyne     MF  37    7   20
9      Cole Palmer  MF,FW  20    1    0


,Player,Pos,MP,Gls,Ast
0,Erling Haaland,FW,37,42,5
1,Jack Grealish,"FW,MF",37,4,8
2,Phil Foden,"FW,MF",36,13,7
3,Riyad Mahrez,"FW,MF",35,12,8
4,Julián Álvarez,FW,35,13,4
5,Rodri,"MF,DF",41,2,6
6,Bernardo Silva,"MF,FW",40,4,5
7,İlkay Gündoğan,MF,38,5,4
8,Kevin De Bruyne,MF,37,7,20
9,Cole Palmer,"MF,FW",20,1,0


In [19]:
def createTweetMsg(df: pd.DataFrame):
    # トップテキストを用意
    top_text = '今季の公式戦スタッツです！' + '\n\n'
    
    # 変数を初期化
    text = top_text
    tweetID = ''
    match_count = 0
    tweet_count = 0
    match_count_sled = 0
    
    for index, row in df.iterrows():
        match_count += 1
        text = text + str(row['Player']) + ' 👕×' + str(int(row['MP'])) + ' ⚽×' + str(int(row['Gls'])) + ' 👟×' + str(int(row['Ast'])) +  '\n\n'
        print(text)
    return

df3 = createTweetMsg(df2)
df3

今季の公式戦スタッツです！

Erling Haaland 👕×37 ⚽×42 👟×5


今季の公式戦スタッツです！

Erling Haaland 👕×37 ⚽×42 👟×5

Jack Grealish 👕×37 ⚽×4 👟×8


今季の公式戦スタッツです！

Erling Haaland 👕×37 ⚽×42 👟×5

Jack Grealish 👕×37 ⚽×4 👟×8

Phil Foden 👕×36 ⚽×13 👟×7


今季の公式戦スタッツです！

Erling Haaland 👕×37 ⚽×42 👟×5

Jack Grealish 👕×37 ⚽×4 👟×8

Phil Foden 👕×36 ⚽×13 👟×7

Riyad Mahrez 👕×35 ⚽×12 👟×8


今季の公式戦スタッツです！

Erling Haaland 👕×37 ⚽×42 👟×5

Jack Grealish 👕×37 ⚽×4 👟×8

Phil Foden 👕×36 ⚽×13 👟×7

Riyad Mahrez 👕×35 ⚽×12 👟×8

Julián Álvarez 👕×35 ⚽×13 👟×4


今季の公式戦スタッツです！

Erling Haaland 👕×37 ⚽×42 👟×5

Jack Grealish 👕×37 ⚽×4 👟×8

Phil Foden 👕×36 ⚽×13 👟×7

Riyad Mahrez 👕×35 ⚽×12 👟×8

Julián Álvarez 👕×35 ⚽×13 👟×4

Rodri 👕×41 ⚽×2 👟×6


今季の公式戦スタッツです！

Erling Haaland 👕×37 ⚽×42 👟×5

Jack Grealish 👕×37 ⚽×4 👟×8

Phil Foden 👕×36 ⚽×13 👟×7

Riyad Mahrez 👕×35 ⚽×12 👟×8

Julián Álvarez 👕×35 ⚽×13 👟×4

Rodri 👕×41 ⚽×2 👟×6

Bernardo Silva 👕×40 ⚽×4 👟×5


今季の公式戦スタッツです！

Erling Haaland 👕×37 ⚽×42 👟×5

Jack Grealish 👕×37 ⚽×4 👟×8

Phil Foden 👕×36 ⚽×13 👟×7

Riyad Mahr

In [ ]:
# スパゲッティ tweet 関数ココおいておく

In [ ]:
from urllib.request import urlopen
import pandas as pd
import time
import re
import tweepy
import requests
from bs4 import BeautifulSoup
from types import MappingProxyType
import datetime
import math
import random
from urllib.request import urlopen
import numpy as np
import modules.settings as Settings

### 定数定義
# ツイートクライアント
client = tweepy.Client(
    bearer_token = Settings.BEARER_TOKEN,
    consumer_key = Settings.CONSUMER_KEY,
    consumer_secret = Settings.CONSUMER_SECRET,
    access_token = Settings.ACCESS_TOKEN,
    access_token_secret = Settings.ACCESS_SECRET
)

# 1ツイートに付与する選手情報数
MAX_PLAYER_INFO = 5

def scrapeFbrefStandardStats():
    # スクレイピング対象のURLを生成する
    url = 'https://fbref.com/en/squads/b8fd03ef/2022-2023/all_comps/Manchester-City-Stats-All-Competitions'
    # https://fbref.com/en/squads/b2b47a98/2022-2023/all_comps/Newcastle-United-Stats-All-Competitions
    
    # スクレイピング
    html = urlopen(url).read().decode('utf-8')

    df = pd.read_html(html)[0]

    # マルチインデックスを解消する
    df = df.reset_index(drop=True)
    
    # 欠損値を含む行を削除する
    df = df.dropna()
    
    # 欠損データパッチ
    df.loc[df['Unnamed: 0_level_0',   'Player'] == 'Kyle Walker', ('Unnamed: 2_level_0', 'Pos')] = 'DF'

    # ポジションを優先順位に従って変換する
    pos_order = {'FW': 0, 'MF': 1, 'DF': 2, 'GK': 3}
    df['Pos'] = df[('Unnamed: 2_level_0', 'Pos')].str.split(',').str[0].str.strip().map(pos_order)

    # ポジションで並べ替える
    df = df.sort_values(by=[('Pos', ''), ( 'Unnamed: 4_level_0',       'MP')], ascending=[True, False]).reset_index(drop=True)

    # 必要なカラムのみを抽出する
    df = df[[( 'Unnamed: 0_level_0',   'Player'),
             ( 'Unnamed: 2_level_0',      'Pos'),
             ( 'Unnamed: 4_level_0',       'MP'),
             (        'Performance',      'Gls'),
             (        'Performance',      'Ast')
            ]]

    # 列名を変更する
    df.columns = ['Player', 'Pos', 'MP', 'Gls', 'Ast']

    # Gls列とAst列を整数型に変換し、小数点以下を切り捨てる
    # 欠損値と無限大の値を除外して整数に変換する
    df['Gls'] = df['Gls'].replace([np.inf, -np.inf], np.nan).dropna().astype(int)
    df['Ast'] = df['Ast'].replace([np.inf, -np.inf], np.nan).dropna().astype(int)

    print(df[:10])

    return df

    
def _tweetBot(msg : str, parentTweetID: str):
    """
    指定されたメッセージをTwitterする関数。
    """
    print('tweetBot start')

    if parentTweetID == '':
        # tweetを実施
        tweet_response = client.create_tweet(
            text=msg
        )
    else:
        # スレッド機能のtweetを実施
        tweet_response = client.create_tweet(
            text=msg,
            in_reply_to_tweet_id=parentTweetID
        )
    print(tweet_response)

    print('tweetBot end')
    return tweet_response.data['id']


def createTweetMsg(df: pd.DataFrame):
    # トップテキストを用意
    top_text = '今季の公式戦スタッツです！' + '\n\n'
    
    # 変数を初期化
    text = top_text
    tweetID = ''
    match_count = 0
    tweet_count = 0
    match_count_in_sled = 0
    sled_count = 0
    last_sled_flag = 0
    
    for index, row in df.iterrows():
        match_count += 1
        text = text + str(row['Player']) + ' 👕×' + str(int(row['MP'])) + ' ⚽×' + str(int(row['Gls'])) + ' 👟×' + str(int(row['Ast'])) +  '\n\n'
        
        # マッチ数が5つ以下なら単一ツイートでOK
        if len(df) <= MAX_PLAYER_INFO:
            # マッチ数 = match_countとなった場合ツイート
            if len(df) == match_count:
                # res = _tweetBot(text, tweetID)
                print('ツイート成功!', res)
            else:
                print('stay...match_count', match_count)
                print('stay...text', text)
        else:
            # 試合カードが5試合たまった場合
            if match_count >= MAX_PLAYER_INFO:
                # １回ツイートする
                # tweetID = _tweetBot(text, tweetID)
                tweet_count += 1
                # テキストとカウンタを初期化
                text = ''
                match_count = 0
                match_count_in_sled = len(df) - MAX_PLAYER_INFO
            else:
                sled_count += 1
                print('sled_count', sled_count)
                
                # スレッド処理を実施
                if match_count_in_sled == match_count:
                    # res = _tweetBot(text, tweetID)
                    print('スレッド処理でツイート成功!', res)
                
                # 最終スレッド処理を実施
                last_sled_flag = len(df) - MAX_PLAYER_INFO - match_count_in_sled * MAX_PLAYER_INFO
                print('len(df)', len(df))
                print('sled_count', sled_count)
                print('MAX_PLAYER_INFO', MAX_PLAYER_INFO)
                print('match_count_in_sled * MAX_PLAYER_INFO', match_count_in_sled * MAX_PLAYER_INFO)
                if last_sled_flag == match_count:
                    # res = _tweetBot(text, tweetID)
                    print('スレッド処理でツイート成功!', res)
                else:
                    print('stay...match_count', match_count)
                    print('stay...text', text)
    return

In [ ]:
# これを改善しないとな

def createTweetMsg(df: pd.DataFrame):
    # トップテキストを用意
    top_text = '今季の公式戦スタッツです！' + '\n\n'
    
    text = top_text
    match_count = 0
    tweetID = ''

    for index, row in df.iterrows():
        match_count += 1
        text = text + str(row['Player']) + ' 👕×' + str(int(row['MP'])) + ' ⚽×' + str(int(row['Gls'])) + ' 👟×' + str(int(row['Ast'])) +  '\n\n'

        # 試合カードが5試合たまった場合
        if match_count >= MAX_PLAYER_INFO:
            # ツイートする
            tweetID = _tweetBot(text, tweetID)
            # マッチカウントを初期化
            match_count = 0

    if match_count > 0:
        # ツイートする
        _tweetBot(text, tweetID)
    return